# Notebook 3 : Entraînement du Modèle CRNN-MFCC

**Projet SereneSense - Détection de Véhicules Militaires par Analyse Audio**

---

## 📋 Table des Matières

1. [Introduction et Contexte](#1-introduction)
2. [Architecture du Modèle CRNN-MFCC](#2-architecture)
3. [Configuration d'Entraînement](#3-configuration)
4. [Processus d'Entraînement](#4-entrainement)
5. [Résultats et Métriques](#5-resultats)
6. [Visualisations et Analyses](#6-visualisations)
7. [Comparaison avec CNN-MFCC](#7-comparaison)
8. [Analyse des Performances Par Classe](#8-analyse-classe)
9. [Conclusion](#9-conclusion)

---


## 1. Introduction et Contexte {#1-introduction}

### Objectif

Ce notebook documente l'entraînement du modèle **CRNN-MFCC** (Convolutional Recurrent Neural Network), une amélioration du modèle CNN-MFCC qui intègre des **couches récurrentes BiLSTM** pour capturer les dépendances temporelles.

### Motivation

Le modèle CNN-MFCC (Notebook 2) a atteint **66.88% de précision** mais souffrait d'overfitting. Le CRNN-MFCC vise à :
- ✅ Améliorer la précision grâce à la modélisation temporelle
- ✅ Utiliser des séquences audio plus longues (4s vs 3s)
- ✅ Capturer les patterns temporels avec BiLSTM
- ✅ Réduire l'overfitting avec une meilleure régularisation

### Résultats Attendus

D'après l'analyse du projet :
- **Best Validation Accuracy** : 73.21% (epoch 47)
- **Final Validation Accuracy** : 72.32% (epoch 100)
- **Nombre de paramètres** : ~1,500,000 (1.5M)
- **Amélioration vs CNN** : +6.33% de précision

---


In [ ]:
# Import des bibliothèques nécessaires
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import yaml
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Chemins du projet
PROJECT_ROOT = Path(r'C:/Users/MDN/Desktop/SereneSense')
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

CONFIG_PATH = PROJECT_ROOT / 'configs' / 'models' / 'legacy_crnn_mfcc.yaml'
HISTORY_PATH = PROJECT_ROOT / 'outputs' / 'history' / 'crnn_baseline.json'
OUTPUT_DIR = PROJECT_ROOT / 'outputs' / 'training_crnn'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Bibliothèques importées avec succès")
print(f"📁 Projet : {PROJECT_ROOT}")
print(f"🔧 PyTorch version : {torch.__version__}")
print(f"🎮 CUDA disponible : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU : {torch.cuda.get_device_name(0)}")


---

## 2. Architecture du Modèle CRNN-MFCC {#2-architecture}

### Structure du Modèle

Le modèle CRNN-MFCC combine convolutions et récurrence :

```
Input: (batch, 3, 40, 124)
  ↓
Conv2D(48, 3×3) → BatchNorm → ReLU → MaxPool(2×1) → Dropout(0.25)
  ↓
Conv2D(96, 3×3) → BatchNorm → ReLU → MaxPool(2×1) → Dropout(0.30)
  ↓
Conv2D(192, 3×3) → BatchNorm → ReLU → MaxPool(2×1) → Dropout(0.30)
  ↓
Reshape → (batch, 124, 960)  # Préserve dimension temporelle
  ↓
BiLSTM(128) → 256 features (bidirectionnel)
  ↓
BiLSTM(64) → 128 features (bidirectionnel)
  ↓
Dual Pooling (Avg + Max) → 256 features
  ↓
Dense(160) → ReLU → Dropout(0.35)
  ↓
Dense(7)
```

### Différences Clés vs CNN-MFCC

| Aspect | CNN-MFCC | CRNN-MFCC |
|--------|----------|-----------|
| **Durée audio** | 3.0 secondes | 4.0 secondes |
| **Input shape** | (3, 40, 92) | (3, 40, 124) |
| **MaxPool** | (2, 2) | (2, 1) - préserve temps |
| **Couches récurrentes** | Aucune | 2× BiLSTM |
| **Paramètres** | 242K | ~1.5M |
| **Modélisation temporelle** | ❌ Non | ✅ Oui |

---


In [ ]:
# Chargement de la configuration
print("📄 Chargement de la configuration CRNN-MFCC...\n")

if CONFIG_PATH.exists():
    with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
        crnn_config = yaml.safe_load(f)
    
    print("🔧 Configuration MFCC :")
    mfcc_cfg = crnn_config.get('mfcc', {})
    for key, value in mfcc_cfg.items():
        print(f"   • {key:20s} : {value}")
    
    print("\n🏗️ Architecture CRNN :")
    crnn_arch = crnn_config.get('crnn', {})
    for key, value in crnn_arch.items():
        print(f"   • {key:20s} : {value}")
    
    print("\n🎨 SpecAugment :")
    spec_aug = crnn_config.get('spec_augment', {})
    for key, value in spec_aug.items():
        print(f"   • {key:20s} : {value}")
else:
    print(f"⚠️ Configuration non trouvée : {CONFIG_PATH}")
    crnn_config = None


In [ ]:
# Définition du modèle CRNN-MFCC (architecture exacte du projet)
class CRNNMFCCModel(nn.Module):
    """Modèle CRNN-MFCC avec BiLSTM pour classification audio."""
    
    def __init__(self, num_classes=7, input_channels=3):
        super(CRNNMFCCModel, self).__init__()
        
        # Blocs Conv (MaxPool préserve dimension temporelle avec (2,1))
        self.conv1 = nn.Conv2d(input_channels, 48, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(48)
        self.pool1 = nn.MaxPool2d((2, 1))
        self.dropout1 = nn.Dropout(0.25)
        
        self.conv2 = nn.Conv2d(48, 96, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(96)
        self.pool2 = nn.MaxPool2d((2, 1))
        self.dropout2 = nn.Dropout(0.30)
        
        self.conv3 = nn.Conv2d(96, 192, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(192)
        self.pool3 = nn.MaxPool2d((2, 1))
        self.dropout3 = nn.Dropout(0.30)
        
        # BiLSTM
        self.lstm1 = nn.LSTM(960, 128, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(256, 64, batch_first=True, bidirectional=True)
        
        # Fully Connected
        self.fc1 = nn.Linear(256, 160)
        self.dropout4 = nn.Dropout(0.35)
        self.fc2 = nn.Linear(160, num_classes)
        
    def forward(self, x):
        # Conv blocks
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(self.pool1(x))
        
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(self.pool2(x))
        
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(self.pool3(x))
        
        # Reshape pour LSTM : (batch, channels, freq, time) → (batch, time, features)
        batch_size, channels, freq, time = x.shape
        x = x.permute(0, 3, 1, 2)  # (batch, time, channels, freq)
        x = x.reshape(batch_size, time, channels * freq)  # (batch, time, 960)
        
        # BiLSTM
        x, _ = self.lstm1(x)  # (batch, time, 256)
        x, _ = self.lstm2(x)  # (batch, time, 128)
        
        # Dual pooling (Average + Max)
        avg_pool = torch.mean(x, dim=1)  # (batch, 128)
        max_pool, _ = torch.max(x, dim=1)  # (batch, 128)
        x = torch.cat([avg_pool, max_pool], dim=1)  # (batch, 256)
        
        # Fully Connected
        x = torch.relu(self.fc1(x))
        x = self.dropout4(x)
        x = self.fc2(x)
        
        return x

# Instanciation du modèle
model = CRNNMFCCModel(num_classes=7, input_channels=3)

# Comptage des paramètres
TOTAL_CNN_PARAMS = 242_000  # Pour comparaison

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("\n🎯 Modèle CRNN-MFCC instancié :")
print(f"   • Paramètres totaux       : {total_params:,}")
print(f"   • Paramètres entraînables : {trainable_params:,}")
print(f"   • Taille du modèle        : {total_params * 4 / 1024 / 1024:.2f} MB (FP32)")
print(f"   • Ratio vs CNN-MFCC       : {total_params / TOTAL_CNN_PARAMS:.1f}×")

# Afficher l'architecture
print("\n📐 Architecture du modèle :\n")
print(model)


In [ ]:
# Test de la forme de sortie
print("🧪 Test de la forme de sortie du modèle :\n")

# Créer un batch d'entrée fictif (4s audio → 124 frames)
batch_size = 4
dummy_input = torch.randn(batch_size, 3, 40, 124)

print(f"   Input shape  : {dummy_input.shape}")

# Forward pass
model.eval()
with torch.no_grad():
    output = model(dummy_input)

print(f"   Output shape : {output.shape}")
print(f"   Expected     : (batch_size={batch_size}, num_classes=7)")
print(f"\n✅ Test réussi !")


---

## 3. Configuration d'Entraînement {#3-configuration}

### Hyperparamètres d'Entraînement

D'après le fichier `legacy_crnn_mfcc.yaml` et les résultats obtenus :

**Optimizer** :
- Type : Adam
- Learning Rate : 1e-3 (0.001)
- Weight Decay : 1e-4 (0.0001) - Régularisation L2
- Betas : (0.9, 0.999)

**Training** :
- Batch Size : 32
- Epochs : 100
- Loss Function : CrossEntropyLoss (avec class weights)
- Audio Duration : 4.0 secondes (vs 3.0s pour CNN)

**Learning Rate Schedule** :
- Type : ReduceLROnPlateau
- Factor : 0.5
- Patience : 10 epochs
- Min LR : 1e-7

**Data Augmentation (SpecAugment)** :
- Frequency Masking : 15% (2 masks)
- Time Masking : 10% (2 masks)
- Probability : 0.8

### Commande d'Entraînement

```bash
python scripts/train_legacy_model.py \
    --model crnn \
    --config configs/models/legacy_crnn_mfcc.yaml \
    --epochs 100 \
    --batch-size 32 \
    --learning-rate 1e-3 \
    --checkpoint outputs/phase1/crnn_improved.pth
```


In [ ]:
# Configuration d'entraînement (pour référence)
training_config = {
    'model': 'CRNN-MFCC',
    'optimizer': 'Adam',
    'learning_rate': 1e-3,
    'weight_decay': 1e-4,
    'batch_size': 32,
    'epochs': 100,
    'loss_function': 'CrossEntropyLoss',
    'lr_scheduler': 'ReduceLROnPlateau',
    'lr_factor': 0.5,
    'lr_patience': 10,
    'num_classes': 7,
    'input_shape': (3, 40, 124),
    'audio_duration': 4.0,
    'sample_rate': 16000,
    'lstm_units': [128, 64],
    'bidirectional': True,
}

print("⚙️ Configuration d'entraînement CRNN-MFCC :\n")
for key, value in training_config.items():
    print(f"   • {key:20s} : {value}")


---

## 4. Processus d'Entraînement {#4-entrainement}

### Détails du Training

L'entraînement a été effectué sur **100 epochs** avec les observations suivantes :

**Convergence** :
- **Meilleure epoch** : 47
- **Best Val Accuracy** : 73.21%
- **Best Val Loss** : ~0.87

**Stabilité** :
- **Overfitting minimal** : Val accuracy reste stable
- **Final Val Accuracy** : 72.32% (epoch 100)
- **Dégradation** : Seulement -0.89% (vs -8.93% pour CNN)

**Amélioration vs CNN-MFCC** :
- +6.33% de précision (66.88% → 73.21%)
- Overfitting divisé par ~10

### Temps d'Entraînement

- **Total** : 3-4 heures sur GPU
- **Par epoch** : ~2-2.5 minutes (plus long que CNN)


---

## 5. Résultats et Métriques {#5-resultats}

### Chargement de l'Historique d'Entraînement

Les résultats d'entraînement sont sauvegardés dans `outputs/history/crnn_baseline.json`.


In [ ]:
# Chargement de l'historique d'entraînement
print("📊 Chargement de l'historique d'entraînement...\n")

if HISTORY_PATH.exists():
    with open(HISTORY_PATH, 'r') as f:
        history = json.load(f)
    
    print("✅ Historique chargé avec succès !\n")
    print(f"📈 Résumé des résultats :")
    print(f"   • Modèle              : {history.get('model')}")
    print(f"   • Epochs demandées    : {history.get('epochs_requested')}")
    print(f"   • Epochs complétées   : {len(history.get('train_loss', []))}")
    print(f"   • Best Val Accuracy   : {history.get('best_accuracy', 0)*100:.2f}%")
    print(f"   • Best Epoch          : {history.get('best_epoch')}")
    print(f"   • Final Train Loss    : {history['train_loss'][-1]:.4f}")
    print(f"   • Final Val Loss      : {history['val_loss'][-1]:.4f}")
    print(f"   • Final Val Accuracy  : {history['val_accuracy'][-1]*100:.2f}%")
else:
    print(f"⚠️ Historique non trouvé : {HISTORY_PATH}")
    print("   Utilisation de données simulées basées sur les résultats connus...\n")
    
    best_epoch = 47
    best_acc = 0.7321
    final_acc = 0.7232
    history = {
        'model': 'CRNN-MFCC',
        'epochs_requested': 100,
        'best_accuracy': best_acc,
        'best_epoch': best_epoch,
        'train_loss': [],
        'val_loss': [],
        'val_accuracy': []
    }
    
    for epoch in range(100):
        if epoch < best_epoch:
            progress = epoch / best_epoch
            history['train_loss'].append(2.0 - progress * 1.3)
            history['val_loss'].append(1.8 - progress * 1.0)
            history['val_accuracy'].append(0.2 + progress * 0.5321)
        else:
            decay = (epoch - best_epoch) / (100 - best_epoch)
            history['train_loss'].append(0.7 - decay * 0.05)
            history['val_loss'].append(0.8 + decay * 0.15)
            history['val_accuracy'].append(best_acc - decay * 0.0089)
    print("✅ Données simulées créées !")


In [ ]:
# Extraction des métriques
train_loss = np.array(history['train_loss'])
val_loss = np.array(history['val_loss'])
val_accuracy = np.array(history['val_accuracy'])
epochs_range = np.arange(1, len(train_loss) + 1)

best_epoch = history['best_epoch']
best_acc = history['best_accuracy']

print(f"\n📊 Statistiques détaillées :\n")
print(f"   Epoch {best_epoch:3d} (Meilleure) :")
print(f"      Train Loss : {train_loss[best_epoch-1]:.4f}")
print(f"      Val Loss   : {val_loss[best_epoch-1]:.4f}")
print(f"      Val Acc    : {val_accuracy[best_epoch-1]*100:.2f}%")
print(f"\n   Epoch {len(train_loss):3d} (Finale) :")
print(f"      Train Loss : {train_loss[-1]:.4f}")
print(f"      Val Loss   : {val_loss[-1]:.4f}")
print(f"      Val Acc    : {val_accuracy[-1]*100:.2f}%")
print(f"\n   📉 Dégradation après best epoch : {(best_acc - val_accuracy[-1])*100:.2f}%")


---

## 6. Visualisations et Analyses {#6-visualisations}

### Courbes d'Entraînement


In [ ]:
# Visualisation des courbes d'entraînement CRNN
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Loss curves
axes[0, 0].plot(epochs_range, train_loss, label='Train Loss', color='steelblue', linewidth=2)
axes[0, 0].plot(epochs_range, val_loss, label='Val Loss', color='darkorange', linewidth=2)
axes[0, 0].axvline(x=best_epoch, color='red', linestyle='--', linewidth=1.5, 
                   label=f'Best Epoch ({best_epoch})')
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Loss', fontsize=12)
axes[0, 0].set_title('Courbes de Loss CRNN-MFCC (Train vs Validation)', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# 2. Validation Accuracy
axes[0, 1].plot(epochs_range, val_accuracy * 100, color='forestgreen', linewidth=2)
axes[0, 1].axvline(x=best_epoch, color='red', linestyle='--', linewidth=1.5,
                   label=f'Best: {best_acc*100:.2f}%')
axes[0, 1].axhline(y=best_acc*100, color='red', linestyle=':', linewidth=1, alpha=0.5)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Accuracy (%)', fontsize=12)
axes[0, 1].set_title('Précision de Validation CRNN-MFCC', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_ylim([0, 100])

# 3. Loss zoom (premiers 60 epochs)
zoom_epochs = min(60, len(epochs_range))
axes[1, 0].plot(epochs_range[:zoom_epochs], train_loss[:zoom_epochs], label='Train Loss', 
                color='steelblue', linewidth=2)
axes[1, 0].plot(epochs_range[:zoom_epochs], val_loss[:zoom_epochs], label='Val Loss', 
                color='darkorange', linewidth=2)
axes[1, 0].axvline(x=best_epoch, color='red', linestyle='--', linewidth=1.5)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Loss', fontsize=12)
axes[1, 0].set_title(f'Zoom: {zoom_epochs} Premières Epochs', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# 4. Overfitting analysis (plus stable que CNN)
gap = val_loss - train_loss
axes[1, 1].plot(epochs_range, gap, color='purple', linewidth=2, label='CRNN Gap')
axes[1, 1].axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.3)
axes[1, 1].axvline(x=best_epoch, color='red', linestyle='--', linewidth=1.5,
                   label='Best Epoch')
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Val Loss - Train Loss', fontsize=12)
axes[1, 1].set_title("Analyse de l'Overfitting (CRNN-MFCC)", fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'crnn_training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Graphique sauvegardé : crnn_training_curves.png")


In [ ]:
# Learning Rate Schedule (simulation)
print("\n📉 Évolution du Learning Rate (ReduceLROnPlateau) :\n")

# Simulation du LR schedule basé sur patience=10
lr_schedule = []
current_lr = 1e-3
plateau_counter = 0
best_val_loss = float('inf')

for epoch in range(len(val_loss)):
    lr_schedule.append(current_lr)
    
    # Vérifier si amélioration
    if val_loss[epoch] < best_val_loss:
        best_val_loss = val_loss[epoch]
        plateau_counter = 0
    else:
        plateau_counter += 1
    
    # Réduire LR si plateau
    if plateau_counter >= 10:
        current_lr *= 0.5
        plateau_counter = 0
        print(f"   Epoch {epoch+1:3d} : LR réduit à {current_lr:.2e}")

# Visualisation
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(epochs_range, lr_schedule, color='crimson', linewidth=2)
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Learning Rate', fontsize=12)
ax.set_title('Évolution du Learning Rate (ReduceLROnPlateau)', 
             fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'crnn_lr_schedule.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💾 Graphique sauvegardé : crnn_lr_schedule.png")


---

## 7. Comparaison avec CNN-MFCC {#7-comparaison}


In [ ]:
# Comparaison CNN vs CRNN
comparison_data = {
    'Métrique': [
        'Best Val Accuracy',
        'Final Val Accuracy',
        'Overfitting (dégradation)',
        'Paramètres',
        'Taille Modèle (MB)',
        'Durée Audio (s)',
        'Input Shape',
        'Temps Entraînement'
    ],
    'CNN-MFCC': [
        '66.88%',
        '57.95%',
        '-8.93%',
        '242,000',
        '0.97 MB',
        '3.0',
        '(3, 40, 92)',
        '2-3h'
    ],
    'CRNN-MFCC': [
        '73.21%',
        '72.32%',
        '-0.89%',
        '1,500,000',
        '6.00 MB',
        '4.0',
        '(3, 40, 124)',
        '3-4h'
    ],
    'Amélioration': [
        '+6.33%',
        '+14.37%',
        '10× mieux',
        '6.2× plus',
        '6.2× plus',
        '+33%',
        '+35% frames',
        '+50% temps'
    ]
}

df_comparison = pd.DataFrame(comparison_data)

print("📊 Comparaison CNN-MFCC vs CRNN-MFCC :\n")
print(df_comparison.to_string(index=False))
print("\n" + "="*80)


In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Précision Best vs Final
models = ['CNN-MFCC', 'CRNN-MFCC']
best_acc_values = [66.88, 73.21]
final_acc_values = [57.95, 72.32]

x = np.arange(len(models))
width = 0.35

axes[0].bar(x - width/2, best_acc_values, width, label='Best Accuracy', 
            color='forestgreen', edgecolor='black')
axes[0].bar(x + width/2, final_acc_values, width, label='Final Accuracy', 
            color='darkorange', edgecolor='black')
axes[0].set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Précision: CNN vs CRNN', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_ylim([0, 100])

for i, (best, final) in enumerate(zip(best_acc_values, final_acc_values)):
    axes[0].text(i - width/2, best + 2, f'{best:.2f}%', ha='center', 
                fontweight='bold', fontsize=10)
    axes[0].text(i + width/2, final + 2, f'{final:.2f}%', ha='center', 
                fontweight='bold', fontsize=10)

# 2. Overfitting (dégradation)
overfitting = [8.93, 0.89]
axes[1].bar(models, overfitting, color=['#d62728', '#2ca02c'], edgecolor='black')
axes[1].set_ylabel('Dégradation (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Overfitting (Best → Final)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

for i, v in enumerate(overfitting):
    axes[1].text(i, v + 0.3, f'{v:.2f}%', ha='center', fontweight='bold', fontsize=11)

# 3. Paramètres et Taille
params = [242000, 1500000]
size_mb = [0.97, 6.00]

ax3_1 = axes[2]
ax3_2 = ax3_1.twinx()

ax3_1.bar([0], [params[0]/1000], 0.35, label='Paramètres (K)', 
                 color='steelblue', edgecolor='black')
ax3_1.bar([1], [params[1]/1000], 0.35, 
                 color='steelblue', edgecolor='black')

ax3_2.bar([0.4], [size_mb[0]], 0.35, label='Taille (MB)', 
                 color='coral', edgecolor='black', alpha=0.7)
ax3_2.bar([1.4], [size_mb[1]], 0.35, 
                 color='coral', edgecolor='black', alpha=0.7)

ax3_1.set_ylabel('Paramètres (K)', fontsize=11, fontweight='bold', color='steelblue')
ax3_2.set_ylabel('Taille (MB)', fontsize=11, fontweight='bold', color='coral')
ax3_1.set_title('Complexité du Modèle', fontsize=14, fontweight='bold')
ax3_1.set_xticks([0.2, 1.2])
ax3_1.set_xticklabels(models)
ax3_1.legend(loc='upper left')
ax3_2.legend(loc='upper right')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'cnn_vs_crnn_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Graphique sauvegardé : cnn_vs_crnn_comparison.png")


---

## 8. Analyse des Performances Par Classe {#8-analyse-classe}

### Résultats Par Classe (Best Model - Epoch 47)

D'après l'analyse du projet, voici les performances par classe :


In [ ]:
# Définition des classes
CLASS_NAMES = [
    'Helicopter',
    'Fighter Aircraft',
    'Military Vehicle',
    'Truck',
    'Footsteps',
    'Speech',
    'Background'
]

crnn_class_metrics = {
    'Class': CLASS_NAMES,
    'Precision': [0.88, 0.95, 0.65, 0.72, 0.68, 0.75, 0.62],
    'Recall': [0.95, 0.52, 0.68, 0.62, 0.58, 0.82, 0.88],
    'F1-Score': [0.91, 0.67, 0.67, 0.67, 0.63, 0.78, 0.73]
}

cnn_class_metrics = {
    'Class': CLASS_NAMES,
    'F1-Score': [0.87, 0.46, 0.51, 0.55, 0.45, 0.66, 0.53]
}

df_crnn = pd.DataFrame(crnn_class_metrics)
df_cnn = pd.DataFrame(cnn_class_metrics)

print("📊 Performances par Classe CRNN-MFCC (Best Model - Epoch 47) :\n")
print(df_crnn.to_string(index=False))

print(f"\n📈 Moyennes pondérées :")
print(f"   • Precision : 0.75 (+0.06 vs CNN)")
print(f"   • Recall    : 0.72 (+0.14 vs CNN)")
print(f"   • F1-Score  : 0.72 (+0.15 vs CNN)")


In [ ]:
# Visualisation des métriques par classe
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

x = np.arange(len(CLASS_NAMES))
width = 0.25

axes[0].bar(x - width, df_crnn['Precision'], width, label='Precision', color='steelblue', edgecolor='black')
axes[0].bar(x, df_crnn['Recall'], width, label='Recall', color='darkorange', edgecolor='black')
axes[0].bar(x + width, df_crnn['F1-Score'], width, label='F1-Score', color='forestgreen', edgecolor='black')
axes[0].set_xlabel('Classe', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Score', fontsize=12, fontweight='bold')
axes[0].set_title('Métriques par Classe (CRNN-MFCC)', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(CLASS_NAMES, rotation=45, ha='right')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_ylim([0, 1.05])

axes[1].bar(x - width/2, df_cnn['F1-Score'], width, label='CNN-MFCC', color='#d62728', edgecolor='black', alpha=0.7)
axes[1].bar(x + width/2, df_crnn['F1-Score'], width, label='CRNN-MFCC', color='#2ca02c', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Classe', fontsize=12, fontweight='bold')
axes[1].set_ylabel('F1-Score', fontsize=12, fontweight='bold')
axes[1].set_title('Comparaison F1-Score: CNN vs CRNN', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(CLASS_NAMES, rotation=45, ha='right')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_ylim([0, 1.05])

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'crnn_class_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Graphique sauvegardé : crnn_class_comparison.png")


In [ ]:
# Analyse détaillée des améliorations par classe
print("\n🎯 Analyse des améliorations par classe (CNN → CRNN) :\n")

for i, class_name in enumerate(CLASS_NAMES):
    cnn_f1 = df_cnn['F1-Score'].iloc[i]
    crnn_f1 = df_crnn['F1-Score'].iloc[i]
    improvement = crnn_f1 - cnn_f1
    improvement_pct = (improvement / cnn_f1) * 100
    emoji = "✅" if improvement > 0.10 else "📈" if improvement > 0.05 else "→"
    print(f"{emoji} {class_name:20s} : {cnn_f1:.2f} → {crnn_f1:.2f} ({improvement:+.2f}, {improvement_pct:+5.1f}%)")

print("\n🔍 Classes les plus améliorées :")
print("   1. Fighter Aircraft : +0.21 (+45.7%) - Meilleure modélisation temporelle")
print("   2. Background       : +0.20 (+37.7%) - Contexte temporel aide la distinction")
print("   3. Military Vehicle : +0.16 (+31.4%) - Patterns temporels mieux capturés")

print("\n⚠️ Classes encore difficiles :")
print("   • Footsteps (F1=0.63) : Sons courts, peu de contexte temporel")
print("   • Background (F1=0.73) : Confusion persiste malgré amélioration")


---

## 9. Conclusion {#9-conclusion}

### Résumé des Résultats CRNN-MFCC

**✅ Points Forts** :
1. **Précision élevée** : 73.21% validation (+6.33% vs CNN)
2. **Stabilité exceptionnelle** : Overfitting minimal (-0.89% seulement)
3. **Modélisation temporelle** : BiLSTM capture les dépendances temporelles
4. **Généralisation** : Final accuracy reste élevée (72.32%)
5. **Amélioration uniforme** : Toutes les classes bénéficient du CRNN
6. **Audio plus long** : 4 secondes capture plus de contexte

**⚠️ Limitations** :
1. **Complexité accrue** : ~1.5M paramètres (6.2× plus que CNN)
2. **Temps d'entraînement** : 3-4 heures (+50% vs CNN)
3. **Taille modèle** : ~6 MB (vs 1 MB pour CNN)
4. **Inférence plus lente** : BiLSTM ajoute de la latence
5. **Classes difficiles** : Footsteps et Background encore problématiques

### Métriques Finales

| Métrique | CNN-MFCC | CRNN-MFCC | Amélioration |
|----------|----------|-----------|-------------|
| **Best Val Accuracy** | 66.88% | 73.21% | **+6.33%** |
| **Final Val Accuracy** | 57.95% | 72.32% | **+14.37%** |
| **Overfitting** | -8.93% | -0.89% | **≈10× mieux** |
| **F1-Score moyen** | 0.57 | 0.72 | **+0.15** |
| **Paramètres** | 242K | 1.5M | 6.2× |
| **Temps entraînement** | 2-3h | 3-4h | +50% |

### Prochaines Étapes

Le **Notebook 4** explorera le modèle **AudioMAE** qui améliore encore les résultats à **82.15%** grâce à :
- Architecture Transformer (Vision Transformer for Audio)
- 111M paramètres (modèle pré-entraîné)
- Mel spectrogrammes 128×128 (vs MFCC)
- 10 secondes d'audio
- Self-attention pour patterns globaux

**Amélioration attendue : +9% (73.21% → 82.15%)**

---

<div style="text-align: center; padding: 20px; background-color: #e8f4f8; border-radius: 10px;">
    <h3>🎉 Notebook 3 Complété !</h3>
    <p><b>CRNN-MFCC : Amélioration Significative avec Modélisation Temporelle</b></p>
    <p>Accuracy : 73.21% | Paramètres : ~1.5M | Durée : 4s | Overfitting : -0.89%</p>
    <p><b>+6.33% vs CNN-MFCC | ~10× moins d'overfitting</b></p>
</div>
